In [1]:
import tensorflow as tf

# 1、共享变量

## 1.1 共享变量的用途

### 在构建模型时，需要使用 tf.Variable() 来创建一个变量。ex：$$biases = tf.Varibale(tf.zeros([2]),name='biases')      # 创建一个偏置的学习参数，在训练时，这个变量不断地更新$$但在某种情况下，一个模型需要使用其他模型创建的变量，两个模型一起训练(ex:GANs)。如果使用 tf.Variable() ,将会生成一个新的变量，但是我们需要的是之前的那个 biases 变量，此时，通过引入 get_variable() 方法，实现共享变量来解决这个问题。

## 1.2 使用 $tf.get\_variable()$ 获取变量

### get_variable 一般会配合 variable_scope (也就是变量作用域)一起使用，以实现共享变量。在某一作用域中的变量可以设置成共享的方式，被其他网络模型使用。
### 下面来介绍 get_variable 的详细使用。其定义如下：$$tf.get_variable(<name>,<shape>,<initializer>)    #详细信息请使用 help 查看$$在TensorFlow里，使用 tf.get_variable 生成的变量是以指定的 name 属性为唯一标识，并不是定义的变量名。使用时一般通过 name 属性定为到具体的变量，并将其共享到其他的模型中

## 1.3 实例1：演示get_variable 和 Variable 的区别

#### 分别使用 Variable 定义变量和使用 get_variable 来定义变量，查看它们的区别

### 1.2.1 Variable 用法演示

#### 其定义如下：$$tf.Variable(<initial-value>, name=<optional-name>)$$

In [2]:
var_1 = tf.Variable(1.0,name = 'first_var')
print('var_1:',var_1.name)
var_1 = tf.Variable(2.0,name = 'first_var')           # 相同的变量名和 name ，不同的取值：1.0,2.0
print('var_1:',var_1.name)

var_2 = tf.Variable(3.0)
print('var_2',var_2.name)
var_2 = tf.Variable(4.0)
print('var_2',var_2.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('var_1:',var_1.eval())
    print('var_2:',var_2.eval())

var_1: first_var:0
var_1: first_var_1:0
var_2 Variable:0
var_2 Variable_1:0
var_1: 2.0
var_2: 4.0


#### 结果分析：上面代码中定义了两次 var_1 ，可以看到内存块中生成了两个 var_1(因为它们具有不同的名字: first_var:0 和 first_var_1:0) ，对于后续的计算来说，第二次定义的 var_1 是生效的(var_1=2.0)；var_2 也表明了：Variable 定义时没有指定名字，系统会自动给加上一个名字 Variable:0

### 1.2.2 get_variable 用法演示

#### 接着上面的代码，使用 get_variable 添加 get_var_1 变量，并运行代码

In [3]:
get_var_1 = tf.get_variable('first_var',[1],initializer=tf.constant_initializer(0.3))
print('get_var_1:',get_var_1.name)
get_var_1 = tf.get_variable('first_var',[1],initializer=tf.constant_initializer(0.4))        # 相同的变量名和 name, 不同的初始值：0.3,0.4
print('get_var_1:',get_var_1.name)

get_var_1: first_var_2:0


ValueError: Variable first_var already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "e:\software\python\python36\lib\site-packages\tensorflow\python\framework\ops.py", line 1654, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "e:\software\python\python36\lib\site-packages\tensorflow\python\framework\ops.py", line 3290, in create_op
    op_def=op_def)
  File "e:\software\python\python36\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


#### 结果分析：可以看到，程序在定义第二个 first_var 时发生了崩溃，这表明：使用 get_variable 只能定义一次指定名称的变量。同时，由于变量 fist_var 在前面使用 Variable 函数生过一次，所以系统自动变成了 first_var_2:0，对上述代码进行修改

In [4]:
get_var_2 = tf.get_variable('second_var',[1],initializer=tf.constant_initializer(0.3))
print('get_var_2:',get_var_2.name)
get_var_2 = tf.get_variable('second_var_1',[1],initializer=tf.constant_initializer(0.4))  # 相同的变量名，不同的 name 属性
print('get_var_2:',get_var_2.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('get_var_2=',get_var_2.eval())

get_var_2: second_var:0
get_var_2: second_var_1:0
get_var_2= [0.4]


#### 结果分析：代码对变量 get_var_2 定义了两次，且其中的 name 各不相同(second_var 和 second_var_1)，此时，代码就没有问题了。同样，在后续的计算过程中，新的 get_var_2 会生效，所以它的输出值是 0.4 而不是 0.3

## 1.4 实例2：在特定作用域下获取变量

#### 在实例1中可知：使用 get_variable 创建两个相同 name 的变量是行不通的，如下代码会报错：
$$get\_var\_3 = tf.get\_variable('third\_var',[2],dtype=tf.float32)$$
$$get\_var\_33 = tf.get\_variable('third\_var',[2],dtype=tf.float32)$$ 
#### 如果真要这么做，可以使用 variable_scope 将它们隔开。ex:

In [5]:
with tf.variable_scope('test_1'):                                 # 定义一个叫 test_1 的作用域
    get_var_3=tf.get_variable('third_var',[2],dtype=tf.float32)
with tf.variable_scope('test_2'):                                 # 定义一个叫 test_2 的作用域
    get_var_33=tf.get_variable('third_var',[2],dtype=tf.float32)

print('get_var_3:',get_var_3.name)
print('get_var_33:',get_var_33.name)

get_var_3: test_1/third_var:0
get_var_33: test_2/third_var:0


#### 结果分析：get_var_3 和 get_var_33 都使用了相同的 name (即 third_var )，通过上述的输出可以看出，其实生成的两个变量get_var_3 和 get_var_33是不同的，它们在不同的 作用域下，这就是 scope 的作用
#### 当然，scope 还支持嵌套，将上述的第二个 with 代码块缩进一下，得到的结果如下

In [7]:
with tf.variable_scope('test_3'):                                 # 定义一个叫 test_3 的作用域
    get_var_4=tf.get_variable('4th_var',[2],dtype=tf.float32)
    
    with tf.variable_scope('test_4'):                                 # 在 test_3 的作用域下，定义一个叫 test_4 的作用域
        get_var_44=tf.get_variable('4th_var',[2],dtype=tf.float32)

print('get_var_4:',get_var_4.name)
print('get_var_44:',get_var_44.name)

get_var_4: test_3/4th_var:0
get_var_44: test_3/test_4/4th_var:0


## 1.5 实例3：共享变量功能的实现

### variable_scope 中有一个参数 reuse ，通过设置 reuse=True 来实现共享变量，表示使用已经定义过的变量。这时，get_variable 将不会再会去创建新的变量，而是去图中找使用 get_variable 创建过的有相同 name 的变量
#### 在实例2中再建立一个同样的 scope ，并且设置 reuse = True ，实现共享 5th_var 变量。ex：

In [8]:
with tf.variable_scope('test_3',reuse=True):                                 # 定义一个叫 test_3 的作用域
    get_var_5=tf.get_variable('4th_var',[2],dtype=tf.float32)
    
    with tf.variable_scope('test_4'):                                 # 在 test_3 的作用域下，定义一个叫 test_4 的作用域
        get_var_55=tf.get_variable('4th_var',[2],dtype=tf.float32)

print('get_var_4:',get_var_4.name)
print('get_var_44:',get_var_44.name)
print('get_var_5:',get_var_5.name)
print('get_var_55:',get_var_55.name)

get_var_4: test_3/4th_var:0
get_var_44: test_3/test_4/4th_var:0
get_var_5: test_3/4th_var:0
get_var_55: test_3/test_4/4th_var:0


#### 结果分析：get_var_4 和 get_var_5 的输出 name 时一样的，get_var_44 和 get_var_55 的输出 name 也是一样的，这表明：get_var_4 和 get_var_5 共用了同一个变量，get_var_44 个 get_var_55 共用了同一个变量，这就实现了变量共享。【只要在 variable_scope 中设置 reuse=True ，则该领域及其子领域中的变量采用了共享模式】
### 在实际应用中，可以把 get_var_4 和 get_var_5 放到一个网络模型里训练，把get_var_5 和 get_var_55 放到另一个网络模型里去训练，而两个模型的训练结果都会作用于同一个模型的学习参数上。

## 1.6 实例4：初始化共享变量的作用域

### variable_scope 和 get_variable 都有初始化的功能。在初始化时，如果没有对当前变量初始化，则 TensorFlow 会默认使用作用域的初始化方法对其初始化。另外，作用域的初始化方法也有继承功能。ex:

In [9]:
with tf.variable_scope('test_6',initializer=tf.constant_initializer(0.4)):                          # 定义一个叫 test_3 的作用域
    get_var_6=tf.get_variable('6th_var',[2],dtype=tf.float32)
    
    with tf.variable_scope('test_66'):                                 # 在 test_3 的作用域下，定义一个叫 test_4 的作用域
        get_var_66=tf.get_variable('6th_var',[2],dtype=tf.float32)
        get_var_666=tf.get_variable('6th_var_1',[2],initializer=tf.constant_initializer(0.3))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())             # initializer 后不加 '()' 会报错
    print('get_var_6=',get_var_6.eval())                    # 作用域 test_6 下的变量
    print('get_var_66=',get_var_66.eval())                  # 作用域 test_66 下的变量
    print('get_var_666=',get_var_666.eval())                # 作用域 test_666 下的变量

get_var_6= [0.4 0.4]
get_var_66= [0.4 0.4]
get_var_666= [0.3 0.3]


#### 结果分析：get_var_6 数组值为 0.4，表明继承了 test_6 的值；get_var_66 数组的值也为 0.4 ，表面其所在定义域也继承了 test_3 的初始化；get_var_666 在创建时同步指定了初始化操作，所以数组值为 0.3

# 2、图

### 前面接触了一些图的概念，其实一个‘图’就代表了一个计算任务，这里系统讲解一下 TensorFlow 的图可以做哪些事情

## 2.1 建立图

### 可以在一个 TensorFlow 中手动建立其他的图，也可以根据图里的变量获得当前的图。下面演示：
#### (1)使用 tf.Graph() 函数建立图
#### (2)使用 tf.get_default_graph() 函数获得图
#### (3)使用 tf.reset_default_graph() 的过程来重置图

In [13]:
c = tf.constant(0.0)           # 定义一个常量 c = 0.0

g = tf.Graph()
with g.as_default():
    c1 = tf.constant(0.0)
    print(c1.graph)
    print(g)
    print(c.graph)

g2 = tf.get_default_graph()
print(g2)

tf.reset_default_graph()
g3 = tf.get_default_graph()
print(g3)

#### 结果分析：
#### (1) c是在刚开始的默认图中建立的，所以图的打印值就是原始的默认图的信息 76CDDDCB38
#### (2) 然后使用 tf.Graph() 函数建立了一个图 76D1C4EBA8，且在新建的图上添加变量 c1 ，可以通过‘variable_name.graph’获得变量的所在图 76D1C4EBA8
#### (3) 在(2)中所建立的新图外，使用 tf.get_default_graph() 函数又获得了原始的默认图 76CDDDCB38，接着使用 tf.reset_default_grpah() 函数，相当于重新建立了一张图来代替原来的默认图，此时的默认图是 76D1C4EDA0

### 【注】：在使用 tf.reset_default_graph() 函数时，必须保证当前图的资源已经全部释放，否则会报错

## 2.2 获取张量

### 在图中可以通过 name 得到其对应的元素。 get_tensor_by_name() 可以获得图里面的张量。ex: 接上述代码继续操作

In [14]:
print(c1.name)
t = g.get_tensor_by_name(name = 'Const:0')
print(t)

Const:0
Tensor("Const:0", shape=(), dtype=float32)


### 结果分析：常量 c1 是在图 g 中建立的。with tf.Graph() as default 代码表示使用 tf.Graph() 函数来创建一个图，并在其上定义 OP；然后将 c1 的 name 放到 get_tensor_by_name() 函数里来反向得到 c1 对应的张量

## 2.3 获取节点

### 获取节点操作 OP 的方法和获取张量的方法很类似，不过此处所用的方法为 get_operation_by_name() 。让我们来比对一下 获取张量和获取节点操作

In [32]:
tf.reset_default_graph()
g3 = tf.get_default_graph()
print(g3)

In [33]:
a = tf.constant([[1.0,2.0]])
b = tf.constant([[1.0],[3.0]])

In [34]:
tensor_1 = tf.matmul(a,b,name='product')
print(tensor_1.name,'****',tensor_1)
test = g3.get_tensor_by_name(name='product:0')         # 获取 tensor 时指定的 name ，格式为：'name:0' or 'name:1', and so on
print(test)
print(''.center(100,'-'))                              # 以 '-' 为分隔符，将打印的内容分隔开

print(tensor_1.op.name)
test_op = g3.get_operation_by_name(name='product')     # 获取 op 时指定的 name ，格式为：'name', 和获取 tensor 不一样
print(test_op)
print(''.center(100,'-'))                              # 以 '-' 为分隔符，将打印的内容分隔开

with tf.Session() as sess:
    test = sess.run(test)
    print(test)
    print(''.center(100,'-'))                              # 以 '-' 为分隔符，将打印的内容分隔开
    
    test = tf.get_default_graph().get_tensor_by_name('product:0')
    print(test)

product:0 **** Tensor("product:0", shape=(1, 1), dtype=float32)
Tensor("product:0", shape=(1, 1), dtype=float32)
----------------------------------------------------------------------------------------------------
product
name: "product"
op: "MatMul"
input: "Const"
input: "Const_1"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "transpose_a"
  value {
    b: false
  }
}
attr {
  key: "transpose_b"
  value {
    b: false
  }
}

----------------------------------------------------------------------------------------------------
[[7.]]
----------------------------------------------------------------------------------------------------
Tensor("product:0", shape=(1, 1), dtype=float32)


#### 【注】：使用默认图时，也可以采用上述代码中的 tf.get_default_graph() 获得当前图，然后再调用 get_tensor_by_name() 函数获取元素
#### 【结果说明】：仔细查看上例中的 OP，通过打印 tensor_1.op.name 的信息，获取了 OP 的名字；然后通过 get_operation_by_name() 函数获得了相同的 OP，可以看出 OP 与 tensor_1 之间的对应关系

## 2.4 获取元素列表

### 如果想查看一下图中的全部元素，可使用 get_operations() 函数来实现。ex:

In [35]:
tt2 = g.get_operations()         # g 中只有一个常量 c1
print(tt2)

[<tf.Operation 'Const' type=Const>]


In [36]:
print(g3.get_operations())

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'product' type=MatMul>]
